In [2]:
freq1, freq2, offsets1, offsets2 = np.random.rand(4, 1000, 1)

In [8]:
np.newaxis

###  단순히 마지막 값을 가져다 쓰기 = > 0.025

In [18]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time- offsets1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 +20))
    series += 0.2 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[np.newaxis].astype(np.float32)

In [13]:
n_steps = 50
series = generate_time_series(10000, n_steps +1)
X_train, Y_train = series[:7000, n_steps], series[:7000, -1]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [9]:
from tensorflow import keras

In [10]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.025799466

###  간단한 RNN 구현

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow

In [18]:
!pip install numpy==1.19.5

  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\users\\owner\\anaconda3\\lib\\site-packages\\~umpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\owner\anaconda3\python.exe -m pip install --upgrade pip' command.


In [19]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True,input_shape =[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])


NotImplementedError: Cannot convert a symbolic Tensor (simple_rnn_7/strided_slice:0) to a numpy array.

### 10개 타임 스텝에 대해서 예측하기

In [ ]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[;, step_ahead])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)
    
y_pred = X[:, n_steps:]


In [19]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
x_test, y_test = series[9000:, :n_steps], series[9000:, -10:, 0]


In [ ]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_squences =True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

Y_pred = model.predict(X_new)

In [9]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss ='mse', optimizer=optimizer, metrics=[last_time_step_mse])


# 층 정규화

In [ ]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation='tanh', **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = keras.layers.LayerNomalization()
        self.activation = keras.activations.get(activation)
        
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer.norm(outputs))
        return norm_outputs, [norm_outputs]
    
    
    
model = keras.model.Sequential([ 
    keras.layers.RNN(LNSimpleRNNCell(20) , return_squences=True, input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_squences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])



# LongTerm Depandancy
# LSTM, GRU

In [2]:
from tensorflow import keras

In [23]:

model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

# RNN 층에 LSTM 설정
model = keras.models.Sequential([
    keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True, input_shape=[None, 1]),
    keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences =True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])




## GRU 예시
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2 , padding='valid', input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss ='mse', optimizer='adam', metrics= [last_time_step_mse])
# 처음 타임 스템 3개 버리고 2 배로 다운 샘플링 => 커널 크기가 4이기 때문에 첫 출력은 0 에서 3까지 타임 스텝으로 만들어짐
history = model.fit(X_train, Y_train[:, 3::2], epochs=20, validation_data=(X_valid, Y_valid[:, 3::2])) 

Epoch 1/20


ValueError: in user code:

    C:\Users\Owner\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\Owner\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Owner\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Owner\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Owner\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    C:\Users\Owner\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:886 __call__
        self.name)
    C:\Users\

In [21]:
X_train

array([[[-1.49332061e-01, -1.97566807e-01,  4.70552444e-02, ...,
          2.09355336e-02, -1.61154732e-01, -2.91074395e-01],
        [ 3.61854643e-01,  1.73219234e-01,  7.73383770e-04, ...,
         -1.24112265e-02, -3.63445908e-01, -4.78524327e-01],
        [-2.65160978e-01, -3.61449629e-01, -2.17099071e-01, ...,
         -4.58453059e-01, -4.16802675e-01, -4.05423760e-01],
        ...,
        [ 4.77962613e-01,  5.70753455e-01,  4.32438195e-01, ...,
         -3.64895046e-01, -3.16522121e-01, -1.23278119e-01],
        [-4.10530925e-01, -1.63921177e-01,  5.59982285e-02, ...,
         -2.16557816e-01, -5.94299212e-02, -1.39205292e-01],
        [ 2.30393156e-01, -1.41779976e-02, -1.94911882e-01, ...,
          7.74508357e-01,  6.31420732e-01,  3.74865860e-01]]],
      dtype=float32)

In [22]:
Y_train

array([[ 0.66938436,  0.38958472, -0.71764076, -0.18001512,  0.0210354 ,
         0.21462297, -0.36819884, -0.04196079, -0.05727043,  0.49721852]],
      dtype=float32)

# WaveNet

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1,2,4,8) *2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding='casual', activation='relu', dilation_rate=rate))
    

model.add(keras.layers.Conv1D(filters=10, kernel_size=1))
model.compile(loss='mse', optimizer='adam', metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid))
